In [1]:
from numba import cuda

In [2]:
import numpy as np

In [21]:
# Need two things - blocks per grid and threads per block

In [6]:
# simple scalar addition

In [3]:
@cuda.jit
def myadd(x, y, z):
    index = cuda.threadIdx.x
    z[index] = x[index] + y[index]

In [4]:
blockspergrid = 1
threadsperblock = 1
X = np.array([1])
Y = np.array([2])
Z = np.array([0])
myadd[1, 1](X, Y, Z)

In [5]:
Z

array([3])

In [6]:
# Vector addition

In [7]:
@cuda.jit
def add_vectors(x, y, z):
    ix = cuda.threadIdx.x
    z[ix] = x[ix] + y[ix]

In [8]:
threadsperblock = 8
blockspergrid = 1

In [10]:
X = np.arange(1, 9)
Y = np.ones((8,))
Z = np.zeros((8,))
add_vectors[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[2. 3. 4. 5. 6. 7. 8. 9.]


In [11]:
# try with a larger vector

In [13]:
X = np.arange(1, 17)
Y = np.ones((16,))
Z = np.zeros((16,))
add_vectors[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[2. 3. 4. 5. 6. 7. 8. 9. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
# What went wrong?

In [14]:
threadsperblock = 16
Z = np.zeros((16,))
add_vectors[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]


In [15]:
# reduce threadsperblock back and change blockspergrid
threadsperblock = 8
blockspergrid = 2
Z = np.zeros((16,))
add_vectors[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[2. 3. 4. 5. 6. 7. 8. 9. 0. 0. 0. 0. 0. 0. 0. 0.]


In [26]:
# what went wrong?

In [16]:
@cuda.jit
def add_blocked_vector(x, y, z):
    ix = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    z[ix] = x[ix] + y[ix]

In [18]:
Z = np.zeros((16,))
add_blocked_vector[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]


In [19]:
# Adding 2d matrices!

In [20]:
X = np.random.randint(0, 10, size=(8, 8))
Y = np.random.randint(0, 10, size=(8, 8))

In [21]:
@cuda.jit
def mat_add(x, y, z):
    ix = cuda.threadIdx.x
    iy = cuda.threadIdx.y
    z[ix, iy] = x[ix, iy] + y[ix, iy]

In [22]:
blockspergrid = 1
threadsperblock = (8, 8)
Z = np.zeros((8, 8))
mat_add[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[[15. 11.  7.  7. 13. 13. 13. 10.]
 [ 5. 14.  7.  9.  8. 13. 14. 10.]
 [11.  2.  5.  9. 13. 14.  7.  9.]
 [12. 10.  5. 11.  1. 11.  2. 12.]
 [ 6. 15. 12. 10. 16.  9. 12. 18.]
 [ 1.  6.  2.  6. 14.  2. 13.  1.]
 [17. 13. 15.  4.  5.  8. 15.  8.]
 [16. 11.  7.  8.  6.  3.  6.  1.]]


In [23]:
print(X + Y)

[[15 11  7  7 13 13 13 10]
 [ 5 14  7  9  8 13 14 10]
 [11  2  5  9 13 14  7  9]
 [12 10  5 11  1 11  2 12]
 [ 6 15 12 10 16  9 12 18]
 [ 1  6  2  6 14  2 13  1]
 [17 13 15  4  5  8 15  8]
 [16 11  7  8  6  3  6  1]]


In [24]:
# try again with 4 by 4 blocks!
blockspergrid = 4
threadsperblock = (4, 4)
@cuda.jit
def mat_add_blocked(x, y, z):
    ix = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    iy = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    z[ix, iy] = x[ix, iy] + y[ix, iy]
Z = np.zeros((8, 8))
mat_add_blocked[blockspergrid, threadsperblock](X, Y, Z)
print(Z.astype(int))

[[15 11  7  7  0  0  0  0]
 [ 5 14  7  9  0  0  0  0]
 [11  2  5  9  0  0  0  0]
 [12 10  5 11  0  0  0  0]
 [ 6 15 12 10  0  0  0  0]
 [ 1  6  2  6  0  0  0  0]
 [17 13 15  4  0  0  0  0]
 [16 11  7  8  0  0  0  0]]


In [37]:
# What went wrong?

In [26]:
blockspergrid = (2, 2)
threadsperblock = (4, 4)
@cuda.jit
def mat_add_blocked(x, y, z):
    ix = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    iy = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    z[ix, iy] = x[ix, iy] + y[ix, iy]
Z = np.zeros((8, 8))
mat_add_blocked[blockspergrid, threadsperblock](X, Y, Z)
print(Z.astype(int))

[[15 11  7  7 13 13 13 10]
 [ 5 14  7  9  8 13 14 10]
 [11  2  5  9 13 14  7  9]
 [12 10  5 11  1 11  2 12]
 [ 6 15 12 10 16  9 12 18]
 [ 1  6  2  6 14  2 13  1]
 [17 13 15  4  5  8 15  8]
 [16 11  7  8  6  3  6  1]]


In [27]:
# Exercise: Try all this with different block, thread config!

In [28]:
# Dot product of 1D vectors

In [29]:
blockspergrid = 1
threadsperblock = 8
@cuda.jit
def mydot(x, y, z):
    ix = cuda.threadIdx.x
    z[0] += x[ix] * y[ix]

In [30]:
X = np.ones((8,))
Y = np.ones((8,))
Z = np.zeros((1,))
mydot[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[1.]


In [68]:
# What went wrong?

In [31]:
temp_Z = np.zeros((8,))
@cuda.jit
def mydot(x, y, temp_z):
    ix = cuda.threadIdx.x
    temp_z[ix] = x[ix] * y[ix]

In [32]:
X = np.ones((8,))
Y = np.ones((8,))
mydot[blockspergrid, threadsperblock](X, Y, temp_Z)
print(temp_Z.sum())

8.0


In [33]:
# Single thread execution
@cuda.jit
def single_thread_dot(x, y, z):
    z[0] = 0.
    for i in range(x.shape[0]):
        z[0] += x[i] * y[i]
blockspergrid = 1
threadsperblock = 1
Z = np.zeros((1,))
single_thread_dot[blockspergrid, threadsperblock](X, Y, Z)
print(Z)

[8.]


In [34]:
# Matrix multiplication!

In [37]:
X = np.random.randint(0, 10, size=(3, 3))
print("X:\n", X)
Y = np.linalg.pinv(X)
print("Y:\n", Y)
print("X * Y:\n", np.dot(X, Y))

X:
 [[7 3 2]
 [3 2 5]
 [7 6 7]]
Y:
 [[ 0.25806452  0.14516129 -0.17741935]
 [-0.22580645 -0.56451613  0.46774194]
 [-0.06451613  0.33870968 -0.08064516]]
X * Y:
 [[ 1.00000000e+00  3.33066907e-16 -1.11022302e-16]
 [-1.11022302e-16  1.00000000e+00  2.77555756e-16]
 [ 3.88578059e-16  2.77555756e-16  1.00000000e+00]]


In [38]:
blockspergrid = 1
threadsperblock = (3, 3)
@cuda.jit
def mat_multiply(x, y, z):
    ix = cuda.threadIdx.x
    iy = cuda.threadIdx.y
    xrow = x[ix, :]
    ycol = y[:, iy]
    for i in range(3):
        z[ix, iy] += xrow[i] * ycol[i]

In [43]:
Z = np.zeros((3, 3))
mat_multiply[blockspergrid, threadsperblock](X, Y, Z)

In [44]:
Z

array([[ 1.00000000e+00,  2.22044605e-16, -1.11022302e-16],
       [-1.11022302e-16,  1.00000000e+00,  2.77555756e-16],
       [ 3.33066907e-16, -1.66533454e-16,  1.00000000e+00]])

In [45]:
# Exercise: Multiply two non-square matrices of size 3, 4 and 4, 5